Originally this was intended for a situation when we want to study the number of females divded by the population size in each time interval.
This script creates a new ratio variable for each group and then performs linear regression on each of these groups.

In [15]:
data_path = "data.csv"

target = ["Number of authors"]
variable = "Year"
groupby = "Gender"

create_CSV = False

save_transforemd_data = True

In [16]:
import pandas as pd

df = pd.read_csv(data_path)

share_dict = {}

for targeted in target:
    total = df.groupby(variable)[targeted].sum()
    for group in df[groupby].unique():
        data = df[df[groupby] == group]
        group_total = data.groupby(variable)[targeted].sum()
        group_ratio = group_total / total 
        share_dict[f"{targeted}_{group}"] = group_ratio
        
share = pd.DataFrame(share_dict)

In [23]:
if save_transforemd_data:
    share.to_csv("reg_share_transformed_data.csv") 

In [6]:
df = share
target = share.columns
variable = variable

In [7]:
from scipy.stats import linregress
from sklearn.metrics import mean_squared_error

def lin_reg(X, y):
    reg = linregress(X,y)

    mse = mean_squared_error(X, reg.slope * y + reg.intercept)

    default = list(reg)
    custom = [reg.intercept_stderr, mse,reg.rvalue**2]

    return default + custom

In [8]:
default_statnames = ["slope", "intercept", "r_value", "p_value", "std_err"]
custom_statnames = ["intercept_stderr", "MSE", "R^2"]
statnames = default_statnames + custom_statnames

result_dict = {}
for targeted in target:
    X = share[targeted]
    y = share.index
    result_dict[targeted] = dict(zip(statnames, lin_reg(X,y)))

result_dict

{'Number of authors_female': {'slope': 178.38127633597128,
  'intercept': 1970.5923061235292,
  'r_value': 0.9939225873993678,
  'p_value': 2.592073542633811e-30,
  'std_err': 3.6070286090896007,
  'intercept_stderr': 0.7298493092483973,
  'MSE': 129396907586.47293,
  'R^2': 0.9878821097426539},
 'Number of authors_male': {'slope': -178.38127633597125,
  'intercept': 2148.9735824595004,
  'r_value': -0.9939225873993678,
  'p_value': 2.592073542633811e-30,
  'std_err': 3.6070286090896,
  'intercept_stderr': 2.9070925148060356,
  'MSE': 126450858169.92769,
  'R^2': 0.9878821097426539}}

In [9]:
result = pd.DataFrame(result_dict).transpose()

if create_CSV:
    result.to_csv("reg_share_results.csv", index=True)

In [10]:
result

,slope,intercept,r_value,p_value,std_err,intercept_stderr,MSE,R^2
Number of authors_female,178.381276,1970.592306,0.993923,2.592074e-30,3.607029,0.729849,1.293969e+11,0.987882
Number of authors_male,-178.381276,2148.973582,-0.993923,2.592074e-30,3.607029,2.907093,1.264509e+11,0.987882
